[![colab-logo](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/japan-medical-ai/medical-ai-course-materials/blob/master/notebooks/Introduction_to_Neural_Network.ipynb)

# ニューラルネットワークの基礎

ここでは，ニューラルネットワーク (Neural Network) についてその概要を紹介していきます．画像認識などに用いられる Convolutional Neural Network (CNN) や，自然言語処理などに用いられる Recurrent Neural Network (RNN) といった手法は，ニューラルネットワークの一種です．

ここではまず，最もシンプルな全結合型と呼ばれるニューラルネットワークの構造について説明を行ったあと，複数の入力データと望ましい出力の組からなる学習用データセットを準備したとき，どうやってニューラルネットワークを学習させればよいのか（教師あり学習の仕組み）について解説を行います．

ニューラルネットワークによって表現される複雑な関数を，現実的な時間で学習するための誤差逆伝播法（バックプロパゲーション）と呼ばれるアルゴリズムについても紹介します．

まずはニューラルネットワークをブラックボックスとして扱ってしまうのではなく，一つ一つ内部で行われる計算を丁寧に調べます．そして，パラメータで特徴づけられた関数で表される線形変換とそれに続く非線形変換を組み合わせて，全体として微分可能な一つの関数を表していることを理解していきます．


## ニューラルネットワークの構造

まずはニューラルネットワークの構造を図式化して見てみましょう．入力変数が{年数，アルコール度数，色合い，匂い}の4変数，出力変数が{白ワイン，赤ワイン}の2変数の場合を示します．

![ニューラルネットワークの基本構造](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/01.png)

この図のひとつひとつの丸い部分のことを**ノード**もしくは**ユニット**と呼び，その縦方向の集まりを**層**と呼びます．そして，一番初めの層を**入力層（input layer）**，最後の層を**出力層（output layer）**，そしてその間を**中間層（intermediate layer）**もしくは**隠れ層（hidden layer）**と呼びます．このモデルは入力層，中間層，出力層の３層の構造となっていますが，中間層の数を増やすことでさらに多層のニューラルネットワークを定義することもできます．この例では各層間の全てのノードが互いに結合されているため，**全結合型のニューラルネットワーク**とも呼び，ニューラルネットワークの最も基礎的な構造です．

入力変数は前章までと同様ですが，出力変数の扱い方がこれまでと異なります．例えば，上図では出力層の各ノードがそれぞれ白ワインと赤ワインに対応しており，カテゴリの数だけ出力の変数があるということになります．なぜこのような構造となっているのでしょうか．

![ニューラルネットワークの出力値](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/02.png)

まず，最終層にどのような値が入るのか，具体例を見てみましょう．例えば，年数が3年物でアルコール度数が14度，色合いが0.2，匂いが0.8で表されるワインがあるとします．内部の計算は後述するとして，このようなデータをニューラルネットワークに与えたときに結果として得られる値に着目してみましょう．上図では，白ワイン $y_{1} = 0.15$, 赤ワイン $y_{2}= 0.85$ となっています．このとき，出力値の中で最も大きな値となっている変数に対応するクラス，すなわち今回の例では「赤ワイン」をこの分類問題におけるこのニューラルネットワークの**予測結果**とすることができます．

ここで出力層のすべての値を合計してみると，1になっていることに気づきます．これは偶然ではなく，そうなるように出力層の値を計算しているためです* ．つまり，出力層のそれぞれのノードが持つ数値は，入力がそれぞれのクラスに属している確率を表していたのでした．そのため，カテゴリ数と同じ数だけ出力層にはノードが必要となります．

それでは，ここからニューラルネットワークの内部で行われる計算を詳しく見ていきましょう．ニューラルネットワークの各層は，前の層の値に線形変換と非線形変換を順番に施すことで計算されています．まずは，ここで言う線形変換とは何を表すのか，から見ていきましょう．

\* 具体的には，Softmax関数という活性化関数（これも後述します）をニューラルネットワークの出力ベクトルに適用することで，出力層における全ノードの値の合計が1になるようにします．

### 線形変換

ここでは，ニューラルネットワークの各層で行われる線形変換について説明します．

![ニューラルネットワークの線形変換](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/linear_transformation.png)

ここで言う線形変換* とは，重み行列 ($w$) $\times$ 入力ベクトル ($h$) $+$ バイアスベクトル ($b$) のような計算のことを指しています．このとき，この変換の入力が$h$，パラメータが$w$と$b$となります．ここでの掛け算（$\times$）は行列の掛け算であることに注意してください．また，これからは，$h$ が文字としてよく登場しますが，これは隠れ層 (hidden layer) の頭文字である $h$ から来ています．ただし，表記を簡潔にするため以下では入力層（上図における $x_1, x_2, x_3, x_4$）を**0層目の隠れ層と考える**ことにして， $h_{01}, h_{02}, h_{03}, h_{04}$ と表記します．では上図で表される計算を数式で記述してみましょう．

（* 通常数学では線形変換とは ${\bf w} \times {\bf h}$ のことを指し，この変換は厳密には「アファイン変換（もしくは アフィン変換）」と呼ばれるものです．しかし，深層学習の文脈ではこの変換も線形変換と呼ぶことも多いです．）

$$
\begin{aligned}
u_{11}&=w_{11}h_{01}+w_{12}h_{02}+w_{13}h_{03}+w_{14}h_{04}+b_{1} \\
u_{12}&=w_{21}h_{01}+w_{22}h_{02}+w_{23}h_{03}+w_{24}h_{04}+b_{2} \\
u_{13}&=w_{31}h_{01}+w_{32}h_{02}+w_{33}h_{03}+w_{34}h_{04}+b_{3}
\end{aligned}
$$

バイアス（$b_1, b_2, b_3$）は上図では省略されていることに注意してください．さて，以上の4つの式は，ベクトルと行列の計算として以下のように書き直すことができます．

$$
\begin{aligned}
\begin{bmatrix}
u_{11} \\
u_{12} \\
u_{13}
\end{bmatrix}&=\begin{bmatrix}
w_{11} & w_{12} & w_{13} & w_{14} \\
w_{21} & w_{22} & w_{23} & w_{24} \\
w_{31} & w_{32} & w_{33} & w_{34}
\end{bmatrix}\begin{bmatrix}
h_{01} \\
h_{02} \\
h_{03} \\
h_{04}
\end{bmatrix}+\begin{bmatrix}
b_{1} \\
b_{2} \\
b_{3}
\end{bmatrix}\\
{\bf u}_{1}&={\bf W}{\bf h}_{0}+{\bf b}
\end{aligned}
$$

本来は ${\bf W}$ や ${\bf b}$ にも，どの層とどの層の間の計算に用いられるものなのかを表す添え字をつけるべきですが，ここでは簡単のため省略しています．

### 非線形変換

次に，非線形変換について説明します．線形変換のみでは，下図右のように入力と出力の間が非線形な関係である場合，両者の間の関係を適切に表現することができません．

![入力と出力の関係](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/04.png)

そこで，ニューラルネットワークでは各層で線形変換に引き続いて非線形変換を施すことで，全体の関数が非線形性を持つようにしています．この非線形変換を行う関数を，ニューラルネットワークの文脈においては **活性化関数** と呼びます．

上図の線形変換の結果 $u_{11}, u_{12}, u_{13}$ に活性化関数を使って非線形変換を行った結果を $h_{11}, h_{12}, h_{13}$ と書き，これらを活性値（activation）と呼びます（下図参照）．これが次の層への入力となります．

![活性値](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/activation.png)

活性化関数の具体例としては，下図に示す **ロジスティックシグモイド関数**（以下シグモイド関数）

![シグモイド関数](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/05.png)

が従来，よく用いられてきました．しかし近年，層数が多いニューラルネットワークではシグモイド関数は活性化関数としてほとんど用いられていません．その理由の一つは，シグモイド関数を活性化関数に採用することで **勾配消失** という現象が起きやすくなり，学習が進行しなくなる問題が発生することがあったためです．これは後で詳述します．これを回避するために，**Rectified Linear Unit (ReLU)** という関数がよく用いられます．これは，以下のような形をした関数です．

![ReLU関数](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/06.png)

ここで， ${\rm max}(0, u)$ は，$0$と$u$を比べて大きな方を返す関数です．すなわち，ReLUは入力が負の値の場合には出力は0で一定であり，正の値の場合は入力をそのまま出力するという関数です．シグモイド関数では，入力が小さな，もしくは大きな値をとった際に，勾配がどんどん小さくなってしまうだろうことがプロットからも見て取れます．それに対し，ReLU関数は入力の値がいくら大きくなっても，一定の勾配が発生します．これがのちほど紹介する勾配消失という問題に有効に働きます．

### 数値を見ながら計算の流れを確認

ここで，下図に書き込まれた具体的な数値を使って，入力 $x_1, x_2, x_3$ から出力 $y$ が計算される過程を確認してみましょう．今は計算を簡略化するためバイアス ${\bf b}$ の計算は省略します（バイアスが全て0であるとします）．数値例として，${\bf x} = \begin{bmatrix} 2 & 3 & 1 \end{bmatrix}^T$ が与えられた時の出力 $y$ の計算手順を一つ一つ追いかけてみましょう．

![出力までの計算例](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/output.png)

前章で解説した重回帰分析では，目的関数のパラメータについての導関数を0とおいて解析的に最適なパラメータを計算できましたが，ニューラルネットワークでは一般的に，解析的にパラメータを解くことはできません．その代わり，この導関数の値（勾配）を利用した別の方法でパラメータを逐次的に最適化していきます．

このため，ニューラルネットワークの場合は，**まずパラメータを乱数で初期化し，ひとまずデータを入力して目的関数の値を計算します**．次にその関数の勾配を計算して，それを利用してパラメータを更新し，その更新後の新しいパラメータを使って再度入力データを処理して目的関数の値を計算し…といったことを繰り返し行っていくことになります．

今，パラメータを初期化した結果，上の図のグラフの枝に与えられているような数値になった状態で，入力層の値に線形変換を施すところまでを考えてみましょう．この計算は，以下のようになります．

$$
\begin{aligned}
u_{11}&=3\times 2+1\times 3+2\times 1=11\\
u_{12}&=-2\times 2-3\times 3-1\times 1=-11
\end{aligned}
$$

次に非線形変換を行う活性化関数としてReLU関数を採用し，以下のように中間層の値を計算してみましょう．

$$
\begin{aligned}
h_{11} &= \max(0, 11) = 11 \\
h_{12} &= \max(0, -11)  = 0
\end{aligned}
$$

同様に，出力層の $y$ の値までを計算すると，

$$
y = 3 \times 11 + 2 \times 0 = 33
$$

となります．

さて，次節からは，パラメータを，どうやって更新していくかを見てみましょう．


## 目的関数

ニューラルネットワークでも，微分可能でさえあれば解きたいタスクに合わせて様々な目的関数を利用することができます．

### 平均二乗誤差

例えば，出力層に$N$個の値を持つニューラルネットワークで回帰問題を解く場合を考えてみましょう．$N$個の出力それぞれ（$y_n (n=1, 2, \dots, N)$）に対して望ましい出力（$t_n (n=1, 2, \dots, N)$）が与えられたとき，目的関数をそれぞれの出力（$y_n$）と対応する正解（$t_n$）の間の **平均二乗誤差（mean squared error）** とすることで，回帰問題を解くことができます．

$$
\mathcal{L} = \dfrac{1}{N} \sum_{n=1}^{N}(t_{n} - y_{n})^{2}
$$

これを最小にするようにニューラルネットワーク中のパラメータを決定するわけです．例えば，上図の例で正解として $t = 20$ が与えられたときの目的関数の値は，

$$
\mathcal{L} = \dfrac{1}{1} (20 - 33)^2 = 169
$$

です．これを小さくするような重み行列の値を探せばよいということです．

### 交差エントロピー

一方，分類問題の場合はしばしば **交差エントロピー（cross entropy）** が目的関数として利用されます．

例として，$N$クラスの分類問題を考えてみましょう．ある入力 $x$ が与えられたとき，ニューラルネットワークの出力層に $N$ 個のノードがあり，それぞれがこの入力が $n$ 番目のクラスに属する確率 $y_n = p(y=n|x)$ を表しているとします．これは，入力 $x$ が与えられたという条件のもとで，予測クラスを意味する $y$ が $n$ であるような確率ということです．

ここで，$x$ が所属するクラスについての正解が， ${\bf t} = \begin{bmatrix} t_1 & t_2 & \dots & t_N \end{bmatrix}^T$ というベクトルで与えられているとします．ただし，このベクトルは $t_n (n=1, 2, \dots, N)$ のいずれか1つだけが1であり，それ以外は0であるようなベクトルであるとします． これを **1-hotベクトル** と呼びます．そして，この1つだけ値が1となっている要素は，その要素のインデックスに対応したクラスが正解であることを意味します．例えば，$t_3 = 1$であれば3というインデックスに対応するクラスが正解であるということになります．

さて，このような準備を行うと，交差エントロピーは以下のように計算できるものとして記述することができます．

$$
\mathcal{L} = - \frac{1}{N} \sum_{n=1}^{N}t_{n}\log y_{n}
$$

#### 補足：交差エントロピーについて

以下は，交差エントロピーの定義について知りたい方だけ参考にしてください．情報理論などで交差エントロピーの定義を知っている方は上の式で表されるものが交差エントロピーとは違うようにみえるかもしれません．しかしこれは，以下のように説明できます．今，$q(y|x)$をニューラルネットワークのモデルが定義する条件付き確率とし，$p(y|x)$を実データの条件付き確率とします．ここで，$p(y|x)$は実際には未知であるため，代わりに学習データの経験分布 

$$
\hat{p}(y|x) = \frac{1}{N} \sum_{n=1}^N I(x =x_n, y=y_n)
$$

を用いることとします．ただし$I$はディラック関数とよばれ，その等号が成立する時，値が$\infty$，それ以外では$0$であるような関数で，その定義域全体にわたる積分は1になるものです．この時，確率分布 $\hat{p}(y|x)$ と $q(y|x)$ の間の交差エントロピーは

$$
\int_{x, y} \hat{p}(y|x) \log \frac{q(y|x)}{\hat{p}(y|x)}   dx dy
$$

と定義されます．ここでディラックのデルタ関数の定義を用い，また$\hat{p}$だけに依存する項を除くと，先程の交差エントロピーの目的関数が導出されます．


## ニューラルネットワークの最適化

目的関数の値を最小にするようなパラメータの値を決定することが，ニューラルネットワークの学習の目的であるとわかりました．では，どのようにしてそのパラメータを探し当てればよいのでしょうか．ある目的関数が与えられたもとで，その目的関数が望ましい値をとるようにニューラルネットワークのパラメータを決定することを，ニューラルネットワークの最適化といいます．

まず最適化の方法を考える前に，まず最適化の対象とはなんであったか，再度確認しましょう．「ニューラルネットワークを最適化する」とは，すなわち「ニューラルネットワークが内部で用いている全てのパラメータの値を適切に決定する」という意味です．では，ニューラルネットワークにおけるパラメータとは，何だったでしょうか．それは，ここまで紹介したシンプルな全結合型ニューラルネットワークの場合，各層の線形変換に用いられていた ${\bf W}$ と ${\bf b}$ のことを指します．

ニューラルネットワークの各パラメータを，目的関数に対する勾配を0とおいて解析的に解くことは，一般的には困難です．しかし，実データをニューラルネットワークに入力すれば，その入力の値における目的関数の勾配を数値的に求めることは可能です．この値が分かれば，パラメータをどのように変化させれば目的関数の値を小さくすることができるのかが分かります．そこで，この勾配を使ってパラメータを繰り返し少しずつ更新していくことで，ニューラルネットワークの最適化を行うことができるのです．この方法について順を追って考えていきましょう．

まず，以下の図を見てください．図中の点線は，パラメータ $w$ を変化させた際の目的関数 $\mathcal{L}$ の値を表しています．この例では簡単のため二次関数の形になっていますが，ニューラルネットワークの目的関数は実際には多次元で，かつもっと複雑な形をしていることがほとんどでしょう．しかし，ここでは説明のためこのようにシンプルな形を想像してみましょう．さて，この目的関数が最小値を与えるような $w$ は，どのようにして発見できるでしょうか．

![パラメータと目的関数の関係（イメージ）](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/13.png)

前節で説明したように，ニューラルネットワークのパラメータはまず乱数で初期化されます．ここでは，例として $w=3$ という初期化が行われたと考えてみましょう．そうすると，$w=3$における$\mathcal{L}$の勾配 $\frac{\partial \mathcal{L}}{\partial w}$ が求まります．ニューラルネットワークの目的関数は，全てのパラメータについて微分可能である* ためです．さて，ここでは仮に $w=3$ における $\frac{\partial \mathcal{L}}{\partial w}$ が $3$ であったとしましょう（このことを$\frac{\partial \mathcal{L}}{\partial w} |_{w=3} = 3$と書きます）．すると，以下の図のように，この $3$ という値は $w=3$ における $\mathcal{L}(w)$ という関数の接線の傾き（勾配; gradient）を表しています．

（* 厳密には損失関数に微分不可能な点が存在する可能性はあります．例えばReLUは $x=0$ の点で微分不可能なため，ReLUを含んだニューラルネットワークには微分不可能な点があります．しかし，通常使うニューラルネットワークの場合，そのような微分不可能な点はわずかしかないため，以下に説明する最適化の方法の中では，無視できます．）

![目的関数の接線の傾き](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/11.png)

傾きとは，$w$ を増加させた際に $\mathcal{L}$ が増加する方向を意味しているので，今は $\mathcal{L}$ の値を小さくしたいわけですから，この傾きの逆方向へ $w$ を変化させる，すなわち $w$ **から** $\partial \mathcal{L} / \partial w$ **を引けばよい** ことになります．

これがニューラルネットワークのパラメータを目的関数の勾配を用いて更新していく際の基本的な考え方です．このときの $w$ のステップサイズ（更新量）のスケールを調整するために，勾配に **学習率 (learning rate)** と呼ばれる値を乗じるのが一般的です．

例えば，今学習率を $0.5$ に設定してみます．そうすると，$w$の更新量は **学習率** $\times$ **勾配** で決まるので，$0.5 \times 3 = 1.5$ となります．現在 $w=3$ なので，**この値を引いて** $w \leftarrow w - 1.5$ と更新した後は， $w=1.5$ となります．上の図は，この1度の更新を行ったあとの状態を表しています．

1度目の更新を行って，$w$ が $w = 1.5$ の位置に移動しました．そこで，再度この点においても勾配を求めてみます．今度は $-1$ になっていたとしましょう．すると **学習率** $\times$ **勾配** は $0.5 \times -1 = -0.5$ となります．これを再び用いて，$w \leftarrow w - (-0.5)$ と2度目の更新を行うと，今度は $w = 2$ の位置にくるでしょう．このようにして，2回更新したあとは，以下の図のようになります．

![パラメータの更新](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/12.png)

徐々に$\mathcal{L}$が最小値をとるときの$w$の値に近づいていっていることが見て取れます．

こうして，**学習率** $\times$ **勾配** を更新量としてパラメータを変化させていくと，パラメータ $w$ を求めたい $\mathcal{L}$ の最小値を与える $w$ に徐々に近づけていくことができます．このような勾配を用いた目的関数の最小化手法を **勾配降下法** と呼びます．ニューラルネットワークは，基本的に **微分可能な関数のみを層間をつなぐ関数として用いて** 設計されるため，登場する関数はすべて微分可能であり，学習データセットを用いて勾配降下法によってパラメータを最適化する方法が適用可能なのです．

ただし，通常ニューラルネットワークを勾配降下法で最適化する場合は，データを一つ一つ用いてパラメータを更新するのではなく，いくつかのデータをまとめて入力し，それぞれの勾配を計算したあと，その勾配の平均値を用いてパラメータの更新を行う方法がよく行われます．これを **ミニバッチ学習** と呼びます．これは，学習データセットから一様ランダムに $k (>0)$ 個のデータを抽出し，その $k$ 個のデータに対する目的関数の平均の値を小さくするようパラメータを更新することを，異なる $k$ 個のデータの組み合わせに対して繰り返し行う方法です．結果的にはデータセットに含まれる全てのデータを使用していきますが，1度の更新に用いるデータは $k$ 個ずつということになります．実際の実装では，データセット内のサンプルのインデックスをまずランダムにシャッフルして並べた配列を作り，その配列の先頭から $k$ 個ずつインデックスを取り出し，対応するデータを使ってミニバッチを構成します．こうして，全てのインデックスを使い切ること，すなわちデータセット内のデータを1度ずつ，すべてパラメータ更新に用い終えることを **1エポックの学習** と呼びます．そして，この $k$ をバッチサイズもしくはミニバッチサイズと呼び，このような学習方法を指して，**確率的勾配降下法 (SGD: Stocastic Gradient Descent)** という名前が用いられます．現在ほとんどすべてのニューラルネットワークの最適化手法はこのSGDをベースとした手法となっています．SGDを用いると，全体の計算時間が劇的に少なくできるだけでなく，下図のように目的関数が凸関数でなかったとしても，適当な条件のもとで“ほとんど確実に”局所最適解に収束することが知られています．

![局所最適解と大域最適解](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/14.png)

### パラメータ更新量の算出

それでは今，下図のような3層の全結合型ニューラルネットワークを考え，1層目と2層目の間の線形変換が ${\bf w}_1, {\bf b}_1$，2層目と3層目の間の線形変換が ${\bf w}_2, {\bf b}_2$ というパラメータによって表されているとします（図ではバイアス ${\bf b}_1, {\bf b}_2$ は省略されています）．また，これらをまとめて $\boldsymbol{\Theta}$ と表すことにします．

![パラメータ更新の例](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/08.png)

入力ベクトルは ${\bf x}$，ニューラルネットワークの出力は ${\bf y} \in \mathbb{R}^N$（$N$ 次元実数ベクトルという意味）とし，入力 ${\bf x}$ に対応した“望ましい出力”である教師ベクトルを ${\bf t}$ とします．ここで，目的関数には前述の平均二乗誤差関数を用いることとしましょう．

さて，パラメータをそれぞれ適当な乱数で初期化したあと，入力 ${\bf x}$ が与えられたときの目的関数の各パラメータについての勾配を計算して，それぞれのパラメータについて更新量を算出してみましょう．

まず，目的関数を改めてベクトル表記を用いて書き下すと，以下のようになります．

$$
\mathcal{L}({\bf y}, {\bf t}) = \frac{1}{N} || {\bf t} - {\bf y} ||_2^2
$$

$|| {\bf t} - {\bf y} ||_2^2$はここでは$({\bf t} - {\bf y})^T({\bf t} - {\bf y})$と同等の意味となります．さらに，ニューラルネットワーク全体を $f$ と書くことにすると，出力 ${\bf y}$ は

$$
\begin{aligned}
{\bf y} &= f({\bf x}; \boldsymbol{\Theta}) \\
&= a_2 ( {\bf w}_2 a_1({\bf w}_1 {\bf x} + {\bf b}_1) + {\bf b}_2 )
\end{aligned}
$$

と書くことができます．ここで，$a_1, a_2$ はそれぞれ，1層目と2層目の，および2層目と3層目の間で線形変換のあとに施される非線形変換（活性化関数）を意味しています．以下，簡単のために，各層間で行われた線形変換の結果を ${\bf u}_1, {\bf u}_2$とし，中間層の値，すなわち ${\bf u}_1$ に活性化関数を適用した結果を ${\bf h}_1$ と書きます．ただし，${\bf u}_2$ に活性化関数を適用した結果は ${\bf y}$ と表記します．すると，これらの関係は以下のように整理することができます．

$$
\begin{aligned}
{\bf y} &= a_2({\bf u}_2) \\
{\bf u}_2 &= {\bf w}_2 {\bf h}_1 + {\bf b}_2 \\
{\bf h}_1 &= a_1({\bf u}_1) \\
{\bf u}_1 &= {\bf w}_1 {\bf x} + {\bf b}_1
\end{aligned}
$$

#### パラメータ ${\bf w}_2$ の更新量

それではまず，出力層に近い方のパラメータ，${\bf w}_2$ についての $\mathcal{L}$ の勾配を求めてみましょう．これは，合成関数の偏微分なので，連鎖律（chain rule）を用いて以下のように展開できます．

$$
\begin{aligned}
\frac{\partial \mathcal{L}}{\partial {\bf w}_2}
&= \frac{\partial \mathcal{L}}{\partial {\bf y}} \frac{\partial {\bf y}}{\partial {\bf w}_2} \\
&= \frac{\partial \mathcal{L}}{\partial {\bf y}} \frac{\partial {\bf y}}{\partial {\bf u}_2} \frac{\partial {\bf u}_2}{\partial {\bf w}_2}
\end{aligned}
$$

この3つの偏微分はそれぞれ，

$$
\begin{aligned}
\frac{\partial \mathcal{L}}{\partial {\bf y}}
&= -\frac{2}{N} ({\bf t} - {\bf y}) \\
\frac{\partial {\bf y}}{\partial {\bf u}_2}
&= \frac{\partial a_2}{\partial {\bf u}_2} \\
\frac{\partial {\bf u}_2}{\partial {\bf w}_2} 
&= {\bf h}_1
\end{aligned}
$$

と求まります．ここで，活性化関数の入力に関する出力の勾配

$$
\frac{\partial a_2}{\partial {\bf u}_2}
$$

が登場しました．これは，例えば活性化関数にシグモイド関数を用いる場合は，

$$
a_2({\bf u}_2) = \frac{1}{1 + \exp(-{\bf u}_2)}
$$

の微分ですから，すなわち

$$
\begin{aligned}
\frac{\partial a_2({\bf u}_2)}{\partial {\bf u}_2}
&= -\frac{-(\exp(-{\bf u}_2))}{(1 + \exp(-{\bf u}_2))^2} \\
&= \frac{1}{1 + \exp(-{\bf u}_2)} \cdot \frac{\exp(-{\bf u}_2)}{1 + \exp(-{\bf u}_2)} \\
&= \frac{1}{1 + \exp(-{\bf u}_2)} \cdot \frac{1 + \exp(-{\bf u}_2) - 1}{1 + \exp(-{\bf u}_2)} \\
&= \frac{1}{1 + \exp(-{\bf u}_2)} (1 - \frac{1}{1 + \exp(-{\bf u}_2)}) \\
&= a_2({\bf u}_2)(1 - a_2({\bf u}_2))
\end{aligned}
$$

となります．シグモイド関数の勾配は，このようにシグモイド関数の出力値を使って簡単に計算することができます．

これで ${\bf w}_2$ の勾配を計算するのに必要な値は全て出揃いました．では実際にNumPyを使ってこれらを計算してみましょう．ここでは簡単のために，バイアスベクトルはすべて0で初期化されているとします．

In [0]:
import numpy as np

# 入力
x = np.array([2, 3, 1])

# 正解
t = np.array([20])

まず，NumPyモジュールを読み込んでから，入力の配列を定義します．ここでは，上図と同じになるように `2, 3, 1` の3つの値を持つ3次元ベクトルを定義しています．また，正解として仮に `20` を与えることにしました．次に，パラメータを定義します．

In [0]:
# 1-2層間のパラメータ
w1 = np.array([[3, 1, 2], [-2, -3, -1]])
b1 = np.array([0, 0])

# 2-3層間のパラメータ
w2 = np.array([[3, 2]])
b2 = np.array([0])

ここでは，以下の4つのパラメータを定義しました．

**1層目と2層目の間の線形変換のパラメータ**

${\bf w}_1 \in \mathbb{R}^{2 \times 3}$ : 3次元ベクトルを2次元ベクトルに変換する行列

${\bf b}_1 \in \mathbb{R}^2$ : 2次元バイアスベクトル

**2層目と3層目の間の線形変換のパラメータ**

${\bf w}_2 \in \mathbb{R}^{1 \times 2}$ : 2次元ベクトルを1次元ベクトルに変換する行列

${\bf b}_2 \in \mathbb{R}^1$ : 1次元バイアスベクトル

それでは，各層の計算を実際に実行してみましょう．

In [0]:
# 中間層の計算
u1 = w1.dot(x) + b1
h1 = 1. / (1 + np.exp(-u1))

# 出力の計算
u2 = w2.dot(h1) + b2
y = 1. / (1 + np.exp(-u2))

print(y)

[0.95257194]


出力は $0.95257194$ と求まりました．つまり，$f([2, 3, 1]^T) = 0.95257194$ ということになります．次に，上で求めた

$$
\frac{\partial \mathcal{L}}{\partial {\bf w}_2}
= \frac{\partial \mathcal{L}}{\partial {\bf y}} \frac{\partial {\bf y}}{\partial {\bf u}_2} \frac{\partial {\bf u}_2}{\partial {\bf w}_2}
$$

の右辺の3つの偏微分をそれぞれ計算してみましょう．

In [0]:
# dL / dy
g_Ly = -2 / 1 * (t - y)

# dy / du_2
g_yu2 = y * (1 - y)

# du_2 / dw_2
g_u2w2 = h1

これらを掛け合わせれば，求めたかったパラメータ ${\bf w}_2$ についての勾配を得ることができます．

In [0]:
# dL / dw_2: 求めたい勾配
g_Lw2 = g_Ly * g_yu2 * g_u2w2

print(g_Lw_2)

[-1.72104507e+00 -1.43112111e-06]


勾配が求まりました．これが $\partial \mathcal{L} / \partial {\bf w}_2$ の値です．これを学習率でスケールさせたものを使えば，パラメータ ${\bf w}_2$ を更新することができます．更新式は，具体的には以下のようになります．

$$
{\bf w}_2 \leftarrow {\bf w}_2 - \eta \frac{\partial \mathcal{L}}{\partial {\bf w}_2}
$$

ここでは学習率を $\eta$ で表記しました．

#### 学習率（learning rate）について

学習率が大きすぎると，繰り返しパラメータ更新を行っていく中で目的関数の値が振動したり，発散したりしてしまいます．小さすぎると，収束に時間がかかってしまいます．そのため，この学習率を適切に決定することがニューラルネットワークの学習においては非常に重要となります．多くの場合，学習がきちんと進むもっとも大きな値を経験的に探すということが行われます．シンプルな画像認識のタスクなどでは大抵，0.1~0.01などが最初に用いられる場合が比較的多く見られます．

#### パラメータ ${\bf w}_1$ の更新量

次に，${\bf w}_1$ の更新量も求めてみましょう．そのためには，${\bf w}_1$ で目的関数 $\mathcal{L}$ を偏微分した値が必要です．これは以下のように計算できます．

$$
\begin{aligned}
\frac{\partial \mathcal{L}}{\partial {\bf w}_1}
&= \frac{\partial \mathcal{L}}{\partial {\bf y}} \frac{\partial {\bf y}}{\partial {\bf w}_1} \\
&=
\frac{\partial \mathcal{L}}{\partial {\bf y}}
\frac{\partial {\bf y}}{\partial {\bf u}_2}
\frac{\partial {\bf u}_2}{\partial {\bf w}_1} \\
&=
\frac{\partial \mathcal{L}}{\partial {\bf y}}
\frac{\partial {\bf y}}{\partial {\bf u}_2}
\frac{\partial {\bf u}_2}{\partial {\bf h}_1}
\frac{\partial {\bf h}_1}{\partial {\bf w}_1} \\
&=
\frac{\partial \mathcal{L}}{\partial {\bf y}}
\frac{\partial {\bf y}}{\partial {\bf u}_2}
\frac{\partial {\bf u}_2}{\partial {\bf h}_1}
\frac{\partial {\bf h}_1}{\partial {\bf u}_1}
\frac{\partial {\bf u}_1}{\partial {\bf w}_1}
\end{aligned}
$$

この5つの偏微分のうち初めの1つはすでに求めました．残りの3つは，それぞれ，

$$
\begin{aligned}
\frac{\partial {\bf y}}{\partial {\bf u}_2}
&= {\bf y}(1 - {\bf y}) \\
\frac{\partial {\bf u}_2}{\partial {\bf h}_1}
&= {\bf w}_2 \\
\frac{\partial {\bf h}_1}{\partial {\bf u}_1}
&= {\bf h}_1(1 - {\bf h}_1) \\
\frac{\partial {\bf u}_1}{\partial {\bf w}_1}
&= {\bf x}
\end{aligned}
$$

と計算できます．では，さっそく実際にNumPyを用いて計算を実行してみましょう．

In [0]:
g_yu2 = y * (1 - y)
g_u2h1 = w2
g_h1u1 = h1 * (1 - h1)
g_u1w1 = x

# 上から du1 / dw1 の直前までを一旦計算
g_Lu1 = g_Ly * g_yu_2 * g_u_2h_1 * g_h_1u_1

# g_u1w1は (3,) というshapeなので，g_u1w1[None]として(1, 3)に変形
g_u1w1 = g_u1w1[None]

# dL / dw_1: 求めたい勾配
g_Lw1 = g_Lu1.T.dot(g_u1w1)

print(g_Lw1)

[[-1.72463398e-04 -2.58695098e-04 -8.62316992e-05]
 [-5.72447970e-06 -8.58671954e-06 -2.86223985e-06]]


これが $\partial \mathcal{L} / \partial {\bf w}_1$ の値です．これを用いて，${\bf w}_2$ と同様に以下のような更新式でパラメータ ${\bf w}_1$ の更新をすることができます．

$$
{\bf w}_1 \leftarrow {\bf w}_1 - \eta \frac{\partial \mathcal{L}}{\partial {\bf w}_1}
$$

## 誤差逆伝播法（バックプロパゲーション）

ここまでで，各パラメータについての目的関数の導関数を手計算により導出して実際に勾配の数値計算を行うということを体験しました．では，もっと層数の多いニューラルネットワークの場合は，どうなるでしょうか．同様に手計算によって導関数を求めることももちろん可能ですが，ニューラルネットワークが微分可能な関数を繰り返し適用するものであるという性質を用いると，コンピュータによって自動的に勾配を与える関数を導き出すことが可能です．合成関数の偏微分は，連鎖律によって複数の偏微分の積の形に変形できたことを思い出しましょう．

下図は，ここまでの説明で用いていた3層の全結合型ニューラルネットワークの出力を得るための計算と，その値を使って目的関数の値を計算する過程を青い矢印で，そして前節で手計算によって行った各パラメータによる目的関数の偏微分を計算する過程を赤い矢印で表現した動画となっています．


![誤差逆伝播法(Backpropagation)の計算過程](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/backpropagation.gif)

まず，目的関数の出力を $l = \mathcal{L}({\bf y}, {\bf t})$ とします．この図の丸いノードは変数を表し，四角いノードは関数を表しています．今，一つの巨大な合成関数として見ることができるニューラルネットワーク全体を $f$ と表し，その中で各層間の線形変換に用いられる関数を $f_1$, $f_2$，非線形変換をそれぞれ $a_1$, $a_2$ と表します．このとき，前節で行った更新量の算出はどのように捉えることができるでしょうか．

今，上図の青い矢印で表されるように，新しい入力 ${\bf x}$ がニューラルネットワークに与えられ，それが順々に出力側に伝わっていき，最終的に目的関数の値 $l$ まで計算が終わったとします．ここまでを **順伝播（forward propagation）** といいます．

すると次は，目的関数の出力の値を小さくするような各パラメータの更新量を求めたいということになりますが，このために必要な目的関数の勾配は，各パラメータの丸いノードより先の部分（出力側）にある関数の勾配だけで計算できることが分かります．具体的には，それらを全て掛け合わせたものになっています．つまり，上図の赤い矢印で表されるように，出力側から入力側に向かって，**順伝播とは逆向きに**，各関数における入力についての勾配を求めて掛け合わせていけば，パラメータについての目的関数の勾配が計算できるわけです．

このように，微分の連鎖律の仕組みを用いて，ニューラルネットワークを構成する関数が持つパラメータについての目的関数の勾配を，**順伝播で通った経路を逆向きにたどるようにして**途中の関数の勾配の掛け算によって求めるアルゴリズムを **誤差逆伝播法（backpropagation）** と呼びます．


## 勾配消失

活性化関数について初めに触れた際，シグモイド関数には勾配消失という現象が起きやすくなるという問題があり，現在はあまり使われていないと説明をしました．その理由についてもう少し詳しく見ていきましょう．

上で既に計算した，シグモイド関数の導関数を思い出してみます．

$$
\begin{aligned}
f\left( u\right) &=\dfrac {1}{1+e^{-u}} \\
f'\left( u\right) &= f\left( u\right) \left( 1-f\left( u\right) \right)
\end{aligned}
$$

さて，この導関数の値を入力変数に関してプロットしてみると，下記のようになります．

![シグモイド関数の導関数](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/09.png)

この図の上2つは，導関数を構成する2つの部分 $f(u)$ と $1 - f(u)$ の値を別々にプロットしたもので，中央下の図が実際の導関数の値となります．上図中央下の導関数の形を見ると，入力が原点から遠くなるにつれ勾配の値がどんどん減少し，0に漸近していくことが見て取れます．

各パラメータの更新量を求めるには，前節で説明したように，**そのパラメータよりも先のすべての関数の勾配を掛け合わせる**必要がありました．このとき，活性化関数にシグモイド関数を用いていると，勾配は必ず**最大でも0.25**という値にしかなりません．すると，線形変換が計算グラフ中に現れるたびに，目的関数の勾配は，多くとも0.25倍されてしまいます．これは，層数が増えていけばいくほど，この最大でも0.25という値が繰り返し掛け合わされることになるため，入力に近い層に流れていく勾配がどんどん0に近づいていってしまいます．

具体例を見てみましょう．今回は3層のニューラルネットワークを用いて説明を行っていましたが，4層の場合を考えてみます，すると，一番入力に近い線形変換のパラメータの勾配は，多くとも目的関数の勾配を $0.25 \times 0.25 = 0.0625$ 倍したものということになります．層数が一つ増えるたびに，指数的に勾配が小さくなるということがよく分かります．

ディープラーニングでは，4層よりもさらに多くの層を積み重ねたニューラルネットワークが用いられます．そうすると，活性化関数としてシグモイド関数を使用した場合，**目的関数の勾配が入力に近い関数が持つパラメータへほぼ全く伝わらなくなってしまいます**，あまりにも小さな勾配しか伝わってこなくなると，パラメータの更新量がほとんど0になるため，どんなに目的関数が大きな値になっていても，入力層に近い関数が持つパラメータは変化しなくなります．つまり初期化時からほとんど値が変わらなくなるということになり，学習が行われていないという状態になるわけです．これを **勾配消失（vanishing gradient）** と呼び，長らく深い（十数層を超える）ニューラルネットワークの学習が困難であった一つの要因でした．

## 「レイヤ」が指すもの

ここまでの解説では，前々節の誤差逆伝播法の説明に登場した図のようなグラフにおける丸いノード（中間出力などの値）の数を指して○層（○-layer）のニューラルネットワーク，という言い方をしてきました．しかし，このグラフにおける四角いノード（関数）を指して層（layer）と呼ぶ場合もあります．そしてニューラルネットワークの実装に使われるフレームワークでは，多くの場合，層タイプ（layer type）として様々な関数がまとめられています．そこで，以下ではこの関数に対して「層」もしくは「レイヤ」という言葉を用いることに注意してください．

## 様々なレイヤタイプ

ここまでで，ニューラルネットワークには大別して線形変換を行う関数と非線形変換を行う関数という二種類の関数が含まれることを説明し，線形変換の例として **全結合層（fully-connected layer）** のみを用いてきました．

しかし，ニューラルネットワークの構成要素として用いることができるレイヤは，全結合層と活性化関数だけではありません．画像認識や画像生成，画像の変換や超解像（低解像の画像から解像度を高めた画像を作る技術）など，様々なタスクで入力に画像というデータ形式が用いられますが，こういったタスクではしばしば画像というデータ形式の特徴と相性の良い変換である畳み込み層（convolution layer）というものが全結合層の代わりにニューラルネットワークの中で線形変換を担います．

また，ニューラルネットワークの構成要素となるレイヤには，微分可能な関数であればなんであろうと用いることができます．そこで，いわゆる **畳み込みニューラルネットワーク（convolutional neural networks; CNN）** と呼ばれるネットワークアーキテクチャの種別では，この畳込み層に加えてプーリング層（pooling layer）というレイヤがしばしば用いられてきました．

本節では，このCNNでしばしば用いられる畳み込み層とプーリング層について，その計算の概要を説明します．

### 畳み込み層（convolution layer）

全結合層では，出力の計算のためにパラメータ行列${\bf W}$中の一つ一つの値が入力データの持つ全ての要素（ベクトルであれば，全次元の値）と掛け合わされていました．言い換えると，「結合」を何らかの演算が行われる関係を指すものとすれば，パラメータ行列${\bf W}$の$i, j$要素である$W_{ij}$と，入力データ${\bf x}$の全要素の間が，全結合（fully-connected）していたということです．

一方，畳み込み層では，ここまでで説明した全結合層とは異なり，パラメータと入力データの間の結合が局所的になっており，各々のパラメータは入力データの全ての要素と結合しているとは限りません．具体的には，2次元畳み込み層の場合，パラメータはカーネル（もしくはフィルタ）と呼ばれる小さな画像パッチのようなものの集合として用意され，そのパッチごとに入力データの一部の値との間の演算が行われる形になっています．

下の図を用いて，$3 \times 3$の大きさの小さな画像パッチのような畳み込みカーネルを2つ持つ畳み込み層の実際の計算過程を調べてみましょう．

![畳み込み層の計算過程](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/convolution.gif)

（図は[CS231n Convolutional Neural Networks for Visual Recognition
](http://cs231n.github.io/convolutional-networks/)より引用）

一番左の青いブロックが並ぶ列は，入力画像を表しています．入力は 幅・高さが$5 \times 5$ の大きさの3チャンネル画像です．上図ではまず，パディング（padding）と呼ばれる処理をして，入力画像の周囲に値が0となる領域を付け足しています．今回は幅1の領域をパディングしています．

中央2列は，それぞれ1列ごとに1つの畳み込みカーネルを表しています．1つの畳み込みカーネルは，入力の画像（もしくは特徴マップ（feature map）と呼ばれる一つ前の層の出力）が持つのと同数のチャンネルを持ちます．ここでは入力画像は3チャンネルだったので，各カーネルは3チャンネル分の値を持っています．これが縦に並んだ3つの赤い$3 \times 3$の大きさのブロックで表されています．各カーネルの各チャンネルは，対応する入力のチャンネルに対して自らをスライドさせていくような形で計算を行います．このときのスライド幅をストライド（stride）と呼びます．図では，2個飛びに入力画像上を$3 \times 3$の枠が移動しているようすが見て取れます．すなわち，ストライドは2に設定されています．ここで行われる計算は，入力のある位置にカーネルを重ね合わせた際に，重なっている$3 \times 3 = 9$個の要素を互いに掛け合わせて，結果を全て足すという計算です．上の図では，例えば1つ目のカーネル（W0）の1つ目のチャンネルは，

$$
\left[
\begin{matrix}
1 & 1 & -1 \\
0 & -1 & 0 \\
1 & -1 & -1
\end{matrix}
\right]
$$

という値をもっており，それに対応する入力画像の1チャンネル目の左上から$3 \times 3$の小領域には

$$
\left[
\begin{matrix}
0 & 0 & 0 \\
0 & 0 & 1 \\
0 & 2 & 0
\end{matrix}
\right]
$$


という値が並んでいます．そのため，それぞれ重なりあっている値同士を掛け合わせると，

$$
\left[
\begin{matrix}
0 & 0 & 0 \\
0 & 0 & 0 \\
0 & -2 & 0
\end{matrix}
\right]
$$

となることが分かります．この9つの値を全て足したものが，まずこのカーネルの1つ目のチャンネルの1つ目の計算結果となります．計算すると，結果は-2です．これを2チャンネル目，3チャンネル目の値でも同様に行うと，2チャンネル目は0，3チャンネル目は1という計算結果になります．そして，最後にこの全てのチャンネルの結果を足し合わせます．その結果は-1となります．これにカーネルごとに用意されるバイアスの値（上図では1つ目のカーネルのバイアスは1）を足したものが，このカーネルの入力の左上隅の領域に対する出力結果となります．結果は，0です．一番右の列の緑色のブロックはこの畳込み層の出力値を表しており，1つ目のブロックの左上のマスを見ると，確かに0となっていることが分かります．

同様に，他のマスの値もいくつか自分で計算してみてください．

このレイヤの微分を考えるようとすると一見複雑に感じられますが，個別のパラメータ値を使って行われる演算は **重みを掛けてバイアスを足す** だけであるため，原理的にはシンプルな線形変換のみで構成されていることからこの変換をパラメータおよび入力について微分することは可能であることが分かります．実際，カーネルサイズが $1 \times 1$ のカーネルを用いた畳み込み演算は各入力次元に同一の重み行列を用いた線形変換を施すのと同等です．たとえば，入力の幅と高さが $W \times H$ で3チャンネルの場合，カーネルサイズが $1 \times 1$ の畳み込み層のカーネルは $1 \times 1 \times 3$ のテンソル，バイアスは1次元のスカラーとなり，$W \times H$ 個の全ての入力次元にこのテンソルとスカラーによる線形変換を施します．

このように，畳み込み層はカーネルを様々に定義することで多様なデータ形式を扱うことができます．1次元のカーネルを使うと1次元の系列データなどにも適用でき，3次元のカーネルを用いれば動画やボクセルのようなデータにも適用できます．

### プーリング層（pooling layer）

プーリングは，主に特徴マップに対して行われる操作で，特徴マップの空間方向の次元（spatial dimension，入力画像における幅・高さに対応する次元）の大きさを削減し計算量を抑えたり，微小な平行移動について出力を不変とすることで画像認識タスクなどにおけるロバスト性の向上のためなどに用いられます．

プーリングの計算方法は畳み込みと似ています．ただしパラメータはなく，畳み込みカーネルを用いて行われた計算の部分が，対応する入力の部分領域に対する平均の計算や最大値計算などに置き換えられたものです．部分領域ごとの平均値を計算していくプーリングを平均値プーリング（average pooling），最大値を計算するプーリングを最大値プーリング（max pooling）と呼びます．

以下の図のうち左側の部分では，プーリング層を用いたダウンサンプリング（解像度を下げること）を $224 \times 224$ サイズの，$64$ チャンネルの特徴マップに適用し，縦横半分の大きさ （$112 \times 112$サイズ）の特徴マップにしています．このとき，チャンネル数（depthとも呼ばれる）は維持されています．

この図のうち右側の部分は，左側の直方体の64チャンネルのうちの一つ，濃い色になっているチャンネルの $4 \times 4$ の部分領域を抜き出してきて模式的に表したものに，$2 \times 2$ 領域ごとに最大値を計算しその領域の代表値とする最大値プーリングを領域を2つ飛びで（ストライド2で）ずらしながら行った際の結果を表しています．

![プーリング層の計算過程](https://github.com/japan-medical-ai/medical-ai-course-materials/raw/master/notebooks/images/3/pooling.png)

（図は[CS231n Convolutional Neural Networks for Visual Recognition
](http://cs231n.github.io/convolutional-networks/)より引用）

